# get the dataset from google Drive

In [1]:
# file_id=1cDieWzU-c_QTXK3U35qV0AoGpxXhCO6r

In [2]:
# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1cDieWzU-c_QTXK3U35qV0AoGpxXhCO6r' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1cDieWzU-c_QTXK3U35qV0AoGpxXhCO6r" -O dataset.rar && rm -rf /tmp/cookies.txt

## unrar the downloaded dataset

In [1]:
# !unrar x dataset.rar 
# !unrar x train.rar
# !unrar x test.rar
# !rm -rf dataset.rar train.rar test.rar
# mkdir dataset && mv test train -t dataset

## take a look at train and validations directories

In [4]:
ls ./dataset/train/

airport_inside/  operating_room/
bakery/          poolinside/
bedroom/         restaurant/
greenhouse/      toystore/
gym/             training_labels.npy
kitchen/         vgg16_Places365_model_features_for_training_images.npy


In [5]:
ls ./dataset/new_val/

airport_inside/  bedroom/     gym/      operating_room/  restaurant/
bakery/          greenhouse/  kitchen/  poolinside/      toystore/


In [6]:
ls ./dataset/new_train/

airport_inside/  bedroom/     gym/      operating_room/  restaurant/
bakery/          greenhouse/  kitchen/  pool_inside/     toy_store/


# import needed libs

In [1]:
import warnings
warnings.filterwarnings('ignore') # for ignoring annoying warnings

In [72]:
import os, glob, pandas as pd
import numpy as np
from PIL import Image
import cv2
from keras import backend as K
from keras.utils import multi_gpu_model
from keras.preprocessing.image import ImageDataGenerator
from Keras_Application_Zoo.vgg16_places_365 import VGG16_Places365 # pretrained model on MIT places 2 dataset 
from keras.optimizers import Adam
from utils import get_classes_weight, to_one_hot#, input_image, get_top_prediction
from keras.callbacks import LearningRateScheduler, CSVLogger, EarlyStopping, ModelCheckpoint, TensorBoard
from keras.utils import multi_gpu_model
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, GlobalAveragePooling2D,GlobalMaxPool2D, Dropout, BatchNormalization, Flatten, Input
from keras.layers.advanced_activations import LeakyReLU
import sklearn
import tensorflow as tf
import matplotlib.pyplot as plt

# model / input configuration parameters

In [82]:
# this parameters dictionary will hold all the static configuration needed
parameters = {
    'input_height' : 224,
    'input_width'  : 224, 
    'input_channel' : 3,
    'input_shape' : (224,224,3),
    'color_mode':'rgb',
    'batch_size'  : 128,
    'nb_classes' : 10,
    'epochs' : 100,
    'Dataset' : './dataset/',
    'train_Dataset' : './dataset/train/',
    'Validation_split': 0.2,
    'validation_Dataset' : './dataset/new_val/',
    'test_Dataset' : './dataset/test/'
}

# Keras Image Datagenerator 
* if we intend to use images instead of extract its features all at once
* won't use it for now we will only use it for getting class weights 

## specifying augmentation parameters

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
#                                    horizontal_flip=True,
                                   validation_split=0.2
                                  )

validation_datagen = ImageDataGenerator(rescale=1./255,
#                                    horizontal_flip=True,
                                  )

## load the images by its labels  

In [5]:
train_generator = train_datagen.flow_from_directory(
    parameters['train_Dataset'],
    target_size=(parameters['input_height'],parameters['input_width']),
    batch_size=parameters['batch_size'],
    shuffle=True,subset='training',
    class_mode='categorical', color_mode =parameters['color_mode']) # set as training data

valid_generator = train_datagen.flow_from_directory(
    parameters['train_Dataset'],
    target_size=(parameters['input_height'],parameters['input_width']),
    batch_size=parameters['batch_size'],subset='validation',
    class_mode='categorical', color_mode =parameters['color_mode']) # set as validation data


Found 2513 images belonging to 10 classes.
Found 625 images belonging to 10 classes.


## calculate the classes weights from generator in case of unblanced data

In [6]:
class_weights = get_classes_weight(train_generator)

In [7]:
class_weights

{0: 1.2076923076923076,
 1: 1.8115384615384615,
 2: 1.1108490566037736,
 3: 7.246153846153846,
 4: 3.1824324324324325,
 5: 1.0,
 6: 5.119565217391305,
 7: 4.205357142857143,
 8: 1.43161094224924,
 9: 2.1216216216216215}

# load [VGG16_Places365](https://github.com/GKalliatakis/Keras-Application-Zoo/blob/master/vgg16_places_365.py) Pretrained model on [MIT PLACES 2](http://places2.csail.mit.edu/) dataset

##  [MIT PLACES 2](http://places2.csail.mit.edu/) dataset have 365 classes and our task have 10 classes only, so we will transfare the learning from [VGG16_Places365](https://github.com/GKalliatakis/Keras-Application-Zoo/blob/master/vgg16_places_365.py) by using it to extract the important features that had been learned from our dataset which is a subset from MIT PLACES2  

In [8]:
vgg16_Places365_model_for_features_extraction = VGG16_Places365(weights='places', include_top=False, input_shape=parameters['input_shape'], pooling='avg')

Instructions for updating:
Colocations handled automatically by placer.


## take a look at model structure

In [9]:
vgg16_Places365_model_for_features_extraction.summary()

Model: "vgg16-places365"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)   

## freeze the VGG16_Places365 Pretrained model so it won't update its weights while transfering learning process 
 * that's in case if we gonna add top layers and flow the data as end to end process

In [10]:
vgg16_Places365_model_for_features_extraction.trainable=False
for i in range(len(vgg16_Places365_model_for_features_extraction.layers)):
    vgg16_Places365_model_for_features_extraction.layers[i].trainable = False

# loop over the training examples and extract the features for each example
* that would save time instead of feeding all data each epoch and extract the features

In [13]:
# implement our image data / label containers
imagesFeaturesList , imageLabelList = [], []

In [14]:
# loop over our train dataset and load the images and labels 
# we can extract features at the same step 
rootdir = parameters['train_Dataset']
for i , (subdir, dirs, files) in enumerate(os.walk(rootdir)):
    for file in files:
        image_file = os.path.join(subdir, file) 
        # for each jpg or png image load it and feed it to the pretrained model and save its features output 
        if(image_file.split('.')[-1].lower() == 'jpg') or (image_file.split('.')[-1] == 'png'):                                        
            image_label = image_file.split('/')[-2]
            image_label_index = data_labels.index(image_label)
            try:
                image = cv2.resize(cv2.imread(image_file), (parameters['input_height'],parameters['input_width']))
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image_as_batch = np.expand_dims(image,axis=0)
                imagesFeaturesList.append(np.squeeze(vgg16_Places365_model_for_features_extraction.predict(image_as_batch),axis=0))
                imageLabelList.append(image_label_index)
            except Exception as e:
                print(e)
                print("something went wrong with ",image_file)


OpenCV(4.2.0) /io/opencv/modules/imgproc/src/resize.cpp:4045: error: (-215:Assertion failed) !ssize.empty() in function 'resize'

something went wrong with  ./dataset/train/poolinside/indooPool_Inside_gif.jpg


# convert the train images features list and train label lists to Numpy arrays so we can train our top layers for classify them to our target classes

In [15]:
imagesFeaturesArray = np.array(imagesFeaturesList)

## convert label array from label index to one hot tensor

In [16]:
imageLabelArray = tf.one_hot(imageLabelList, parameters['nb_classes'])  # output: [3 x 3]


## convert tf one hot tensor to numpy array again 

In [18]:
imageLabelArray
sess = tf.Session()
with sess.as_default():
    imageLabelArray = imageLabelArray.eval()

# shuffle our training dataset 

In [30]:
imagesFeaturesArray, imageLabelArray = sklearn.utils.shuffle(imagesFeaturesArray, imageLabelArray)


## save extracted features and corrosponding labels for later use or continue training 

In [22]:
np.save('./dataset/train/vgg16_Places365_model_features_for_training_images.npy', imagesFeaturesArray)
np.save('./dataset/train/training_labels.npy', imageLabelArray)

# add our top layers to predict withen the 10 classes we have

In [23]:
# specify input shape
input_layer = Input(shape=imagesFeaturesArray.shape[1:])
# add first fully connected layer
fc1 = Dense(512, activation='relu')(input_layer)
# add drop out layer to avoid overfitting
fc1_dropout = Dropout(rate=0.5)(fc1)
# add third and final fully connected layer
fc3 = Dense(parameters['nb_classes'], activation='softmax')(fc1_dropout)
# form our top layers model
_10_scenes_classification_model= Model(input=input_layer, output=fc3)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## take a look for top layers summary

In [24]:
_10_scenes_classification_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                5130      
Total params: 267,786
Trainable params: 267,786
Non-trainable params: 0
_________________________________________________________________


## compile the model with the desired optimization algorithm and loss function

In [25]:
# _10_scenes_classification_model_multiGPU = multi_gpu_model(_10_scenes_classification_model, gpus=4)
_10_scenes_classification_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(0.001),
              metrics=['accuracy'])

### train the top layers/ model on extracted features from the Places pretrained model

In [31]:
history = _10_scenes_classification_model.fit(
    x=imagesFeaturesArray,
    y= imageLabelArray,
    batch_size=parameters['batch_size'],
    validation_split=0.2,
    class_weight=class_weights,
    workers=16,
    use_multiprocessing=True,
    shuffle=True,
#     steps_per_epoch=imagesFeaturesArray.shape[0] // parameters['batch_size'],
    epochs=parameters['epochs'],
    callbacks=[ModelCheckpoint('./weights/_10_scenes_classification_model_TOPLAYERONLY.h5',save_best_only=True)])

Train on 2509 samples, validate on 628 samples
Epoch 1/100
2509/2509 [==============================] - 0s 41us/step - loss: 0.1480 - acc: 0.9904 - val_loss: 0.2093 - val_acc: 0.9889
Epoch 2/100
2509/2509 [==============================] - 0s 36us/step - loss: 0.1120 - acc: 0.9888 - val_loss: 0.2504 - val_acc: 0.9889
Epoch 3/100
2509/2509 [==============================] - 0s 33us/step - loss: 0.1749 - acc: 0.9853 - val_loss: 0.2032 - val_acc: 0.9920
Epoch 4/100
2509/2509 [==============================] - 0s 33us/step - loss: 0.1226 - acc: 0.9872 - val_loss: 0.1636 - val_acc: 0.9920
Epoch 5/100
2509/2509 [==============================] - 0s 32us/step - loss: 0.1157 - acc: 0.9868 - val_loss: 0.1309 - val_acc: 0.9904
Epoch 6/100
2509/2509 [==============================] - 0s 32us/step - loss: 0.1338 - acc: 0.9900 - val_loss: 0.2057 - val_acc: 0.9873
Epoch 7/100
2509/2509 [==============================] - 0s 34us/step - loss: 0.0647 - acc: 0.9908 - val_loss: 0.2210 - val_acc: 0.9889
E

2509/2509 [==============================] - 0s 30us/step - loss: 0.0198 - acc: 0.9964 - val_loss: 0.1285 - val_acc: 0.9920
Epoch 61/100
2509/2509 [==============================] - 0s 31us/step - loss: 0.0410 - acc: 0.9956 - val_loss: 0.1288 - val_acc: 0.9889
Epoch 62/100
2509/2509 [==============================] - 0s 32us/step - loss: 0.0164 - acc: 0.9980 - val_loss: 0.1311 - val_acc: 0.9889
Epoch 63/100
2509/2509 [==============================] - 0s 30us/step - loss: 0.0119 - acc: 0.9980 - val_loss: 0.0976 - val_acc: 0.9920
Epoch 64/100
2509/2509 [==============================] - 0s 32us/step - loss: 0.0149 - acc: 0.9976 - val_loss: 0.1205 - val_acc: 0.9904
Epoch 65/100
2509/2509 [==============================] - 0s 33us/step - loss: 0.0227 - acc: 0.9964 - val_loss: 0.1286 - val_acc: 0.9920
Epoch 66/100
2509/2509 [==============================] - 0s 32us/step - loss: 0.0436 - acc: 0.9944 - val_loss: 0.1868 - val_acc: 0.9873
Epoch 67/100
2509/2509 [==============================

# create the final model formed from freezed pretrained model and our top layers

In [32]:
# _10_scenes_classification_model = Model(input=vgg16_Places365_model_for_features_extraction.input, output=fc3)
final_10_scenes_classification_model = Sequential()
final_10_scenes_classification_model.add(vgg16_Places365_model_for_features_extraction)
final_10_scenes_classification_model.add(_10_scenes_classification_model)

## take a look at our new arch

In [33]:
final_10_scenes_classification_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16-places365 (Model)      (None, 512)               14714688  
_________________________________________________________________
model_1 (Model)              (None, 10)                267786    
Total params: 14,982,474
Trainable params: 267,786
Non-trainable params: 14,714,688
_________________________________________________________________


# save the final Model Weights

In [60]:
final_10_scenes_classification_model.save('./weights/final_10_scenes_classification_model.h5')

# predict test images and save the result in csv format

In [70]:
# save the test results
files_names = []
files_classes = []

## test the final model on test data 

In [71]:
for image_file in glob.glob('./dataset/test/*.jpg'):
    try:
        test_image = cv2.cvtColor(cv2.resize(cv2.imread(image_file),(parameters['input_width'],parameters['input_height'])), cv2.COLOR_BGR2RGB)
        test_image_as_batch = np.expand_dims(test_image, axis=0)
        predictions = final_10_scenes_classification_model.predict(test_image_as_batch)[0]
        top_preds = np.argsort(predictions)[::-1][0]
            
        files_names.append(image_file.split('/')[-1])
        files_classes.append(data_labels[top_preds])
    except Exception as e:
#         print(e, image_file)
        files_names.append(image_file.split('/')[-1])
        files_classes.append('cannot be loaded! ')

## save the result in dataframe then to csv format using pandas

In [75]:
data = {'image file name':  files_names,
        'class': files_classes}

result_dataframe = pd.DataFrame (data, columns = ['image file name','class'])


In [78]:
result_dataframe.to_csv('./test_result.csv')